### Stereotype Detection and Scoring
This note book employs a large language model to detect and score instances of stereotypes in writing. The context and guidelines around stereotypes are provided by Africa No Filter (ANF).

In [1]:
!python --version
# This notebook runs on python version 3.9.18

Python 3.9.18


#### Setup
We load a user writing sample and define a very simple RAG query engine that we’ll use to evaluate using top-k retrieval. First set up the enviironment.

In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

##### Load the required packages  
Note: Chroma is a lightweight, in memory vector store.
Some other vector stores are hosted in the cloud and are used to persist large stores of data

In [3]:
#!pip install chromadb
from langchain.document_loaders import PyPDFLoader #not compatible with python3.8
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma 
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [4]:
# Use the most appropriate language model from OpenAI
#llm_name = "gpt-3.5-turbo"
llm_name = "gpt-4-1106-preview"
print(llm_name)

gpt-4-1106-preview


#### Load the user document

In [5]:
# Load the PDF
loaders = [
    PyPDFLoader("data/Africans are somewhat.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

##### Split the document 

In [6]:
# Split the documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
len(splits)

1

##### Define the type of embeddings we'll be using

In [7]:
embedding = OpenAIEmbeddings()

##### Set up the vector database

In [8]:
persist_directory = 'data/chroma/'
# Tidy up: Forcefully and recursively delete the ./data/chroma directory and all its contents 
# without asking for confirmation. 
!rm -rf ./data/chroma  

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

1


##### Set up the language model

In [9]:
llm = ChatOpenAI(model_name=llm_name, temperature=0) #we want factual answers

#### Bring in the ANF-provided examples and explanations around stereotypes as contect for the LLM

The urls for the reference examples are as follows:  
  
[Bias Example 1 - UNICEF](https://www.unicef.org/southafrica/stories/giving-back-community-humanitarian-response)  
[Bias Example 2 - World Food Program](https://twitter.com/WFPChief/status/1358510664822759436)  
[Bias Example 3 - ActionAid Zambia](https://actionaid.org/news/2021/actionaid-zambia-mining-companies-must-not-be-allowed-pollute-impunity)  
  
[Ethical Example 1 - CARE](https://www.care.org/news-and-stories/health/how-community-activists-in-kenya-are-working-together-to-end-female-genital-cutting/)  
[Ethical Example 2 - AFDB](https://www.afdb.org/en/news-and-events/press-releases/chad-african-development-bank-grants-1126-million-women-and-girls-education-40195)  
[Ethical Example 3 - Open Society Foundation](https://www.opensocietyfoundations.org/what-we-do/regions/africa/)  


**Notes**  
a - The explanation of each example, good and bad, is taken from the ANF document, "How to tell an Africqn story".  
b - The transcript of the WFP twitter video is obtained using Twitter Video Downloader (to download the video), Audacity (to extract the audio), and Descript (to transcribe the audio).   
c - The link to Ethical Example 3 appears to have changed. Therefore this example is not used.

##### Examples illustrating bias

In [10]:
bias_example_1 = """
Giving back to the community - a humanitarian response\
Humanitarian heroes take up the call to improve the welfare of others. \
Giving back to the community is their mandate, working to bring about sustainable social reform.\

The world around us harbours much disparity, inequality and poverty, which calls for a response.\
Humanitarian organizations are involved in the background and on the frontlines, concerned with \
improving the welfare of others and taking up the call to help alleviate suffering. \
Giving back to the community is their mandate - actively working to bring about change and \
social reform. That is a big ask!\

When it comes to giving back to the community, the integrity of a humanitarian organization’s \
mission is at stake. The litmus test for humanitarian authenticity is simple:  \
there is no place for prejudice on grounds of race, gender, sexual orientation, religious, \
national or ethnic divisions. The goal is to save lives, promote the wellbeing of those less \
fortunate, relieve suffering, and maintain human dignity. \
This is a huge but necessary task!\

A personal humanitarian call\
Giving back to the community, however, begins with individual heroes. Empathy, \
compassion and concern for others are part of our humanity. \
Mandela Day offers each one of us, as South Africans, a timeous opportunity to heed \
Madiba’s call, ‘Our society needs to re-establish a culture of caring,’ which starts with each \
beating heart - no matter age, gender or ethnicity.\

Caring is what makes us human, and caring for others is what enables us to change the world.\
This is not idealism, it is simply seeing the need of a fellow human being, and being \
prepared to help and make a meaningful difference.\

Each one of us has the power and opportunity to positively impact the life of another. \
It could be a one-off donation, a monthly pledge donation, volunteering our time, or \
offering resources or services. That is what giving back to the community looks like - it’s \
all about an ‘I can help’ mindset and attitude. \

An organization’s humanitarian response\
Formal charity organizations and established NPOs all contribute in giving back to the \
community in ways that you and I, as individuals, do not have capacity or means to do. \
Large humanitarian organizations have the infrastructure, partnerships and resources \
to not only offer relatively large-scale relief solutions but include a philanthropic vision \
towards sustainable solutions and working to improving social problems in the long term.\

In a country like South Africa, immediate areas of focus generally include hygiene and \
sanitation, health and nutrition, education, and protection of people’s rights. \
Offering relief and positive change in these basic areas involve ongoing financial donor \
support, advocacy and resources.\

That is what humanitarian organizations are equipped to do. \
By partnering with individuals, giving back to the community becomes a meaningful exercise. \
Charity and humanitarian organizations also play a huge role in being ready to step in and \
provide shorter-term disaster/emergency relief and crisis care situations including natural \
disasters and pandemics.\ 

UNICEF South Africa’s humanitarian mission\
UNICEF South Africa is a humanitarian organization that focuses on improving children’s \
lives and the communities in which they live through advocacy, social initiatives and \
emergency relief programmes. Protecting the rights of every child, and helping to \
reimagine a better future for each child, is what makes UNICEF South Africa a \
humanitarian hero. 
"""



In [11]:
explanation_bias_example_1 = """
UNICEF asks for donations because it is 'heroic'.\
In a blog post subtitled 'Humanitarian heroes take up the call to improve the welfare of \
others',UNICEF depicts humanitarian organisations and people who donate to charities\
as 'heroes'. This downplays the role of local heroes who bring about change in their own\
communities and ignores local people’s own agency.\

It argues that, without the intervention of humanitarian organisations, action is unfeasible:\
Formal charity organisations and established NPOs all contribute in giving back to the \
community in ways that you and I, as individuals, do not have capacity or means to do.\
The purpose of the blog post is to raise funds for UNICEF, so it blows the UNICEF trumpet. \
Even though it mentions 'partnering with individuals', the only other people depicted in the \
post are in stock images of happy, black, voiceless children.\

On the one hand, the article emphasises a common humanity by saying, \
'Caring is what makes us human'; on the other hand it argues that those who 'care' are \
somehow uniquely heroic when they undertake acts of charity. \

Thus, the givers are depicted as heroic but not the people in need, acting for themselves.\
Furthermore, the article uses sweeping examples to explain and illustrate the problems of \
South Africans. Unique problems become diluted into generalised 'African' problems, \
like 'hygiene and sanitation, health and nutrition, education, and protection of people’s \
rights'. The meanings underpinning the story signal that Africans are unclean, hungry, \
uneducated, and unable to act to protect their own rights. \

The article thus focuses on the communities’ supposed deficits, rather than the assets \
they can use to change their circumstances.
"""

In [12]:
bias_example_2 = """
Here I am in Mek'ele in a warehouse, a government warehouse in the world food program. \
We're bringing in about 20, 000 metric tons of food that will support about 1. 3 million \
people and the millions of people in need here in the Tigray region because of the conflict. \
And we're trying to address and help every single person get the food they need in this very \
difficult time.\
You can see they're unloading the trucks as we speak because people are hungry and we're moving \
the supplies out there. We're bringing the trucks in, doing what's necessary. As you stand here \
in this warehouse, you can see, uh, they're stacking it up and we'll redistribute this food, \
this wheat out into distribution points all throughout the Tigray region.\
And so we're looking forward to scaling up. We'll be, we're looking forward to supporting \
an additional million people. The government will be supporting about two and a half million \
people working with the NGOs. We're going to make certain that every single person in this \
region, in the Tigray, are going to get the food they need to be healthy.
"""

In [13]:
explanation_bias_example_2 = """
World Food Programme distributing food\
On 7 February 2021, the World Food Programme (WFP) chief, David Beasley, appeared in\
a tweeted video, explaining what WFP is doing about food shortages in Tigray, Ethiopia. \
Throughout the video the camera focuses exclusively on the “white saviour” who has come to \
deliver food. Despite being presented as an authority on the food shortage, he does not explain \
why people in Tigray need food aid, nor the source of the conflict. \

The aim of the video is not to explain what is happening in Tigray, but merely to promote \
WFP’s work. The Ethiopians depicted in the video are in the background: we do not see their \
faces, and they do not speak; they are just manual labour, moving white bags of food aid around.\ \
Although the WFP chief briefly mentions that organisations on the ground are delivering food, \
he does not highlight who these organisations are, how they will reach the neediest people \
and where the people of Tigray can access food.
"""

In [14]:
bias_example_3 = """
Konkola Copper Mines – a subsidiary of the UK-based mining giant Vedanta – has been polluting \
the main water source of surrounding villages in Chingola, in Zambia’s Copperbelt Province, \
and leaving a trail of human rights and environmental abuses for nearly two decades. \ 

A legal action, brought by more than 2,500 Zambian villagers, including 643 children, against \
Konkola Copper Mines Plc and Vedanta Resources Limited, has settled out of court, it was \
announced today (19 January) in a joint statement from the mining companies and Leigh Day, \
the English law firm representing the claimants.  \

ActionAid Zambia works closely with communities in the Chingola area through its partner, \
the Catholic Diocese of Ndola. This work provides a platform and resources to support \
villagers to claim their rights and advocate for regulation of corporate conduct through \
progressive policies and legislation.  \

Mining companies often avoid paying corporation tax in Zambia and are not being held \
accountable to national environmental laws. \ 

Responding to the settlement announcement, Nalucha Nganga Ziba, Country Director of ActionAid \
Zambia, says: 'Communities in Chingola have been fighting for justice and compensation for the \
human rights abuses and environmental destruction they have suffered for over a decade.  This \
case highlights the devastating and deadly impact that polluters like KCM and Vedanta are \
having on surrounding villages in the Copperbelt. \ 

Extractive companies must be taxed on their profits, to provide public services to benefit \
communities, and be made to adhere to environmental standards. There must be no bailouts or \
public investments in mining companies, which continue to pollute and contribute to climate \
change. They must no longer be allowed to act with impunity.'  

‘Our children are dying’ – extracts from interviews with community activists and claimants \
Esther and Brenda:  

Esther Nalumbulu is from Shimulala where the community’s main water source has been polluted \
for more than a decade by toxic run-off from the Konkola Copper Mine (KCM). She says this is \
causing adults to suffer from upset stomachs and skin irritations. But for young children and \
babies it is proving deadly.  \

'We are always ill,' she says. 'Many children have died. Initially when we noticed a high rise \
in the deaths of children in the community, we didn’t realise that it was the water killing \
the children. Unfortunately, the problem has become worse even to this day.' \  

Esther, a community activist who has received rights training from ActionAid Zambia, says \
mothers are advised to keep feeding their babies breast milk for as long as possible and to \
not give them the local water until they’re at least six months old.\  

If babies do drink the water, Esther says: 'The child’s physical health immediately starts to \
change, and you would even find that the child [could] die.'\   

The copper mine in Nchanga has been contaminating the local water supply and polluting \
farmland since 2004. Esther has letters dating from 2011, from a doctor hired by the mine \
itself, explaining that the water is not safe for human consumption and that the community \
is advised not to drink from the stream.\  

Pollution from the mine has contaminated the local water table. Toxins are killing off crops, \
livestock and the vegetation surrounding the river, destroying the livelihoods of local people \
who depended on farming and fishing to support their families.  \

Women and children are most affected as they are forced to walk long distances making 10km \
round trips in search of clean water. The older and disabled people unable to walk so far, \
get sick from drinking the polluted water. Many people are affected by skin conditions due \
to bathing in the river.\

'Women suffer most because they have to fetch water from the streams in the bush,' Esther, \
explaining they have to walk at least 5-6km to find clean water.\

Community activist Brenda Siachipula, 39, is also from Shimulala and has received training on \
rights from ActionAid Zambia. She uses social media and appears on local radio stations to \
raise awareness of the pollution and the impacts it’s having on the community’s health and \
livelihoods.\

'If you go to the mine right now, you will find that they are still releasing the chemical \
slurry into the stream,' she says. 'And when you pass by the steam, the fumes are suffocatingly \
strong and that is an indication that the mine is operating.'  

She says life is a struggle for her community. The pollution has affected the fertility of \
the soil and is killing livestock.\  

'The water the cattle are drinking is killing them, just as it is killing the people,' Brenda \
says. 'When the cows give birth, the calf will die shortly afterwards. Even though the land is \
no longer fertile, we try our best to grow as much as we can, so we can sustain our lives \
and our family’s.'\ 

Brenda says families are struggling to pay school fees for their children: 'This pollution has \
destroyed so many things. There is no point where we can say we are ok. Things are getting \
worse. We are also failing to educate our children.'\  

She adds: 'In my lifetime I want to see that the community is compensated.' 
"""

In [15]:
explanation_bias_example_3 = """
ActionAid describes community action, but relies on pity\

In a story on 19 January 2021, ActionAid described court action undertaken by a Zambian \
community to address problems resulting from mining activity in their area. \

The article initially starts off well, describing the community’s solidarity and agency:\
Communities in Chingola have been fighting for justice and compensation...\
However, midway through the article, ActionAid implies that the community could not have \
fought without having 'received rights training from ActionAid'. It thereby positions local \
activists as learners who receive education from outside rather than equal partners who \
share knowledge.\

The second part of the article also quotes female activists who speak about their woes, \
rather than their strengths and activities. While the specific problems do need to be \
identified, this focus on deficits leaves the audience with feelings of pity for the \
women and their plight, instead of admiration for the women’s actions. \
Action Aid could have inspired the audience to support these empowered women by showing their \
assets – strong organising and problem-solving skills.
"""

##### Examples illustrating ethical writing

In [16]:
ethical_example_1 ="""
How Community Activists in Kenya are Working Together to End Female Genital Cutting\
Kenya banned FGM in 2011, but the practice continues. Due to poor enforcement of the law, \
community activists have made it their mission to save girls from this harmful tradition.\
Shinina Shani can’t remember the last time she turned her cell phone off.\
'Everyone has my number, and when I miss a call from a number I don’t recognize, I have to call \
them back,'' she says. There are literally lives on the line.\

Shinina, 45, has spent the last 20 years as an activist against female genital mutilation or \
cutting (FGM/C). She shares her number with girls in her hometown of Narok, Kenya, who seek \
support as they attempt to avoid “being cut.” She receives about 20 calls a week.\

FGM, sometimes known as female circumcision, is the cultural practice of cutting some or all \
parts of the external female genitalia. In some communities, it’s seen as a rite of passage \
that signifies womanhood and is sometimes followed shortly by marriage. Kenya banned FGM in \
2011, but the practice continues, and its prevalence ranges from 1 percent to 98 percent, \
depending on region and ethnic community. According to the national bureau of statistics, \
FGM is practiced at a rate of 78 percent among the Maasai, the ethnic community that is the \
majority of the population in Narok. Due to poor enforcement of the law, Kenyans like Shinina \
have made it their mission to save girls from this harmful tradition.\

Growing up, Shinina witnessed FGM’s devastating impacts in her community.\
'I’ve seen the complications from delivering children. I’ve seen women not get out of bed for \
three months. I’ve seen them bleed and die,” she says.
This has fueled her activism, which keeps her busiest at the end of the year.
'I don’t rest from November. December is our worst month.'

Since schools in Kenya close for the two-month holiday, FGM spikes as parents believe girls \
will have time to recover from the procedure. In the weeks that follow, girls may experience \
excessive bleeding, severe pain, and infections, and in the long-term, they can face problems \
with urination and menstruation, complications during childbirth, and psychological trauma. \
One WHO study that examined the mental health of women who underwent female genital mutilation \
showed that 30 percent of survivors had post-traumatic stress disorder and 80 percent had \
anxiety disorders.And so she leaves her phone on — for the girls who will inevitably call for \
help.\

Sometimes, the girls don’t call, but they show up.\
One day I just came home and found a child [outside my house], and then I made calls to \
alert authorities and get her to a rescue center,” she says. Another time, she found a girl \
hiding behind her car, too scared to go home.Shinina says that people in Narok used to hold \
lavish parties during the holidays to celebrate girls’ circumcisions. 'It used to be a big \
ceremony with a lot of music. People would witness [a girl being cut] and stand by her and \
give her presents.'

She says some villages in Narok still publicly celebrate like this, but most of these \
ceremonies were driven underground when Kenya banned FGM.\
This has led girls like Faith to live in fear. When she was 12, Faith heard her parents \
discuss plans to circumcise her over the holidays. She knew it was dangerous from \
conversations with girls who’d undergone the procedure.\

'They said you’ll be bleeding for four days. They said it’s a wound and it won’t heal that \
fast, so I was scared,' Faith says.That December, she decided to run away. She confided in a \
teacher, who made the four-hour drive to drop her off at House of Hope Rescue Centre, one of \
four centres in Narok that takes in girls escaping FGM or early child marriage.\

Patrick Ngigi, a 47-year-old pastor and former teacher started the center in 1997 when he was \
inundated with requests by girls who needed support.'They would come to my office to ask for \
help. I realized we needed a place we could keep them [safe].” The first three girls he helped \
hide from their families stayed at his mother’s house. But he was unable to keep up with the \
demand and decided to open a support center, which he runs independently.\

The House of Hope accepts girls throughout the year but is busiest during the holidays. \
When Shinina needs a safe place to house girls, even if it’s in the middle of the night, \
she reaches out to Patrick.\

'We make them feel at home as much as possible. We give them counselling and we ensure they \
continue attending school,' Patrick says. He finds donors who help cover the cost of enrolling \
the girls in boarding schools. During the holidays, they return to House of Hope. \
Patrick explains that if the girls attend day schools, their relatives may 'snatch them' while \
they’re on the way to school and make them undergo FGM.\

House of Hope is currently home to 67 girls, who sleep in bunk beds in dormitory rooms. \
All of them have found their way there voluntarily.\

'Every girl that comes to us is a girl who wants to be there by herself,'' Patrick says. \
'If we try [to bring them here], we will end up getting killed.'\

The center’s goal is to provide a safe space for girls and eventually reunite them with their \
parents. When a girl arrives at the center, Patrick begins a reconciliation process with the \
parents, which also involves elders, chiefs and children’s aid workers. \
The aim is to have parents sign-off on a document declaring they will not force their \
daughter to undergo FGM so that she can safely return home.\

Patrick is currently working to reconcile 26 girls with their parents, a process that can \
take anywhere from a few months to years.This was the case with Faith. After running away \
to the House of Hope, she began the reconciliation process with her parents but it wasn’t \
going well. For years, they told her that if she returned home, they would force her to undergo FGM, \
so she stayed put, completing her elementary school, high school and university studies with \
the support of the rescue center. After graduating from university at age 20, her parents were \
convinced of the dangers of FGM and finally signed off on the reconciliation paperwork.\

Although it took nearly a decade, Faith is certain that she has broken the cycle in her family. \
Her older sister underwent FGM, but her parents assure her that her younger sister won’t.\
Shinina says Faith’s story is one of many that demonstrates the impact of community activists. \
'We have many stories. We’ve rescued girls who were about to be married off, we’ve rescued girls \
who were about to go through FGM. Now they’re professionals, got degrees, or run their own \
families,'' she says. 'There’s a lot of success in it. It’s not advocacy in vain.'
"""

In [17]:
explanation_ethical_example_1 = """
Care International highlighting African agency
In a Care International blog, entitled “How community activists in Kenya are working together \
to end female genital cutting”, the author, Jacky Habib, puts at the centre of the story \
African voices and organisations involved in the work of ending female circumcision. \

The story focuses on one Kenyan woman activist, Shinina Shani, who has been working to stop \
genital cutting. However, Habib also brings in other role-players, naming individuals involved \
and one of the shelters where work is being done. She then emphasises resources and assets \
available in the communities where Shinina works.\

The article also explains that the scale of the problem is different in different parts of \
Kenya, so it does not treat the country as a homogenous whole. The story is nuanced and \
provides various statistics alongside powerful narratives about people who take action.\
Furthermore, Habib provides details about the multi-pronged approach that the African \
role-players have adopted, including providing shelter, counselling and access to education for \
girls and arranging meetings with parents, elders and chiefs.\

The story is ethical and effective because, while it does not shy away from the ongoing \
problem of genital cutting in Kenya, it also does not show Kenyans as hopeless and in need of \
white saviours.
"""

In [18]:
ethical_example_2 = """
Chad: The African Development Bank grants $11.26 million for women and girls’ education\
The Board of Directors of the African Development Bank has approved a grant of $11.26 million \
to the Government of Chad to finance the Girls' Education and Women's Literacy Project– the \
first Bank grant exclusively targeting women and girls.\

The project will be financed from the Bank's Transition Support Facility and will be \
implemented over a five-year period by the Chadian Ministry of National Education and \
Civic Promotion, in coordination with partners involved in the education sector, civil society \
organisations, and youth organisations. The Chadian government will contribute a non-monetary \
contribution of $713,000 towards the program.\

'Through this financing, the African Development Bank is providing support to the Chadian \
government to reduce inequalities through access to education - especially for girls. \
This enables the development of job skills and the improvement of women's productivity \
potential through literacy, job training and the development of income-generating activities,' \
said Solomane Koné, the Bank’s Deputy Director for Central Africa.\

The Project aims to help improve access to quality secondary education in a safe and healthy \
school environment for 5,000 girls as well as train 2,200 teachers and administrative \
officials. It is also expected to provide literacy programs to more than 7,500 women in Chad’s \
Hadjer Lamis, Ouaddaï and N'Djaména regions, The Bank-funded project has a component to raise \
awareness among target-area residents about reducing incidence of gender-based violence, \
as well as on the importance of girls' schooling to reduce early marriage and pregnancy. \
In Chad, 67% of girls are married before the age of 18, and 30% of girls are married before \
the age of 15, according to non-governmental organization Girls Not Brides.\

The Girls' Education and Women's Literacy Project plans to renovate or rebuild school buildings \
and institutions, such as the Amriguébé school complex in N'Djamena that educates pre-primary, \
primary and high school children, and a new women's high school in Massakory, Hadjer Lamis \
region, which would receive educational, scientific and digital equipment. Project components \
have provisions to help supply both schools with safe drinking water, solar power, school \
clinics, build girl-friendly latrines, as well as establish computer and science laboratories.

The Girls' Education and Women's Literacy Project is integrated into the Government of Chad’s \
Interim Education Plan which is working to upgrade the nation’s education system and strengthen \
human capital –education, health and well-being of children and youth today who will become \
Chad’s working population of tomorrow.
"""

In [19]:
explanation_ethical_example_2 = """
African Development Bank providing grants to Chad\
In a press release on 8 January 2021, the African Development Bank (AfDB) announced a new \
grant to Chad, which would be used to provide education and job skills to women and girls. \
Because AfDB provided the grant, the article reveals that African financial institutions \
support African countries; it is not just Western development agencies that can make a \
difference. The press release explains:\

The project will be financed from the Bank’s Transition Support Facility and will be \
implemented over a five-year period by the Chadian Ministry of National Education and \
Civic Promotion, in coordination with partners involved in the education sector, civil \
society organisations, and youth organisations. The Chadian government will contribute a \
non-monetary contribution of $713,000 towards the program.\

The press release therefore shows how the grant fits in with needs on the ground, in terms of \
the Chadian government’s own agenda; working with local schools to boost the integration of \
girls and facilities for them; and working with local civil society organisations and youth \
organisations to ensure programme delivery.\

The press release emphasises cooperation and collaboration, rather than conflict. \
This is markedly different from many communications by development organisations, which \
underscore conflict.
"""

In [20]:
explanation_ethical_example_3 = """
Kenyan innovator developing COVID-19 app\
On 4 February 2021, the Open Society Foundation for Eastern Africa shared an article about a \
Kenyan innovator, Brian Ndegwa, who had invented the Rona app to give Kenyans access to \
COVID-19 health information, up-to-date statistics by Kenyan county, a symptoms checklist for \
users to see if they might have COVID-19, and the possibility of ordering suitable medicine \
to relieve symptoms.\

Because the article highlights African innovation, it addresses negative narratives in a \
direct way. While the article asks for donations and support, it makes it clear that the funds \
should flow directly to the app developer and not go to a development agency.\

The article shows that Africans are deft at technological innovation and can handle necessary \
funds without relying solely on development organisations. It emphasises local solutions to \
local problems.
"""

 ##### Combine the pieces of context

In [21]:
context = f"""
## Examples of biased writing and their explanations
{bias_example_1}
{explanation_bias_example_1}
{bias_example_2}
{explanation_bias_example_2}
{bias_example_3}
{explanation_bias_example_3}

## Examples of ethical writing and their explanations
{ethical_example_1}
{explanation_ethical_example_1}
{ethical_example_2}
{explanation_ethical_example_2}
{explanation_ethical_example_3}
"""

#### Build the prompt

In [22]:
stereotype_template = """
Use the following context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

You are an expert evaluation system for a question answering chatbot.
Your task is to detect and score instances of stereotyping.

You are given the following context:
- Examples illustrating stereotypes. These are examples of biased writing.
- An explanation of each biased example.
- Examples illustrating the lack of stereotypes. These are examples of ethical writing.
- An explanation of each ethical example.

Your job is to judge whether the user writing sample contains stereotypes.
Output a single score that represents a holistic evaluation.
You must return your response in a line with only the score.
Do not return answers in any other format.
On a separate line provide your reasoning for the score.
Also provide the user with recommendations for improvement.

Follow these guidelines for scoring:
- Think step by step and quote every biased or ethical sentence in your reasoning.
- Your score has to be between 1 and 5, where 1 is the worst and 5 is the best.
- Give a numeric score and nothing else.

Scoring Criteria:
- Score of 1: The writing sample is very closely aligned with one or more of the biased \
examples and contains clear, overt stereotypes.
- Score of 2: The writing sample contains several elements of bias or stereotypes, but they \
are less blatant than those warranting a score of 1. It shows some alignment with biased \
examples but also some distance.
- Score of 3: The writing sample is somewhat neutral or mixed, containing neither clear bias \
nor fully ethical writing. It may have minor stereotypes or could be interpreted in different \
ways. This score reflects ambiguity or a balance between biased and ethical examples.
- Score of 4: The writing sample is mostly free from stereotypes and aligns more closely \
with ethical examples. Any biases are subtle and not overtly stereotypical.
- Score of 5: The writing sample has no stereotypes and is very closely aligned with one or \
more of the ethical examples. It represents ethical writing.

End your reasoning with concrete, actionable recommendations to improve the document.
Take the time to carefully craft and review your response to ensure it aligns with the question. Only provide a response when you are 100% confident that you have done the task correctly.

{context}
Question: {question}
Helpful Answer:
"""


##### Set up the Stereotype chain prompt and the QA chain

In [23]:
stereotype_chain_prompt = PromptTemplate.from_template(stereotype_template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True, #allows us to inspect the dics we retrieve
    chain_type_kwargs={"prompt": stereotype_chain_prompt}
)

In [24]:
qa_chain?

##### Run the QA chain with the question

In [25]:
question = "Does the document have any stereotypes?"
result = qa_chain({"query": question})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [26]:
# Print the result dictionary
print(result)

{'query': 'Does the document have any stereotypes?', 'result': '2\n\nThe writing sample contains several elements of bias or stereotypes. The phrases "Africans are somewhat hard working" and "A few Africans I’ve met are lazy" both generalize a diverse continent\'s population, implying that being hardworking or lazy is a trait that can be broadly applied to Africans. The statement "Some Africans are good dancers" also perpetuates a stereotype by suggesting that being a good dancer is a notable characteristic of Africans, even though it is framed as a positive trait.\n\nRecommendations for improvement:\n- Avoid generalizations about any group of people. Instead of attributing traits to an entire continent\'s population, focus on individuals or specific communities and avoid making broad statements.\n- Use a more diverse range of examples to illustrate the complexity and variety within any group.\n- If discussing traits or behaviors, provide context that does not rely on stereotypes or ge

Print final results for the client

In [27]:
# Define a print_results function that extracts and prints the Score and Reasoning 
# from the 'result' key
def print_results(result):
    score, reasoning = result['result'].split('\n', 1)
    print(f"Score: {score} out of 5")
    # Check if "Reasoning" is already present at the start of the reasoning string
    if reasoning.strip().startswith("Reasoning:"):
        # If it is, print the reasoning as is
        print(f"{reasoning}\n")
    else:
        # If "Reasoning" isn't there, add it
        print(f"Reasoning: {reasoning}\n")


In [28]:
print_results(result=result)

Score: 2 out of 5
Reasoning: 
The writing sample contains several elements of bias or stereotypes. The phrases "Africans are somewhat hard working" and "A few Africans I’ve met are lazy" both generalize a diverse continent's population, implying that being hardworking or lazy is a trait that can be broadly applied to Africans. The statement "Some Africans are good dancers" also perpetuates a stereotype by suggesting that being a good dancer is a notable characteristic of Africans, even though it is framed as a positive trait.

Recommendations for improvement:
- Avoid generalizations about any group of people. Instead of attributing traits to an entire continent's population, focus on individuals or specific communities and avoid making broad statements.
- Use a more diverse range of examples to illustrate the complexity and variety within any group.
- If discussing traits or behaviors, provide context that does not rely on stereotypes or generalizations. For example, instead of saying 

References:  
[Building Evaluation from Scratch](https://docs.llamaindex.ai/en/v0.8.37/examples/low_level/evaluation.html)  
[Langchain: Chat with your data](https://www.deeplearning.ai/short-courses/)